<a href="https://colab.research.google.com/github/Omar-Works/ID-2024/blob/main/basiccnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Data Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Malaria/train',  # Path to train data
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # For binary classification (Infected vs Uninfected)
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Malaria/val',  # Path to validation data
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Building the CNN Model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the 3D data to 1D
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting

# Output Layer
model.add(Dense(1, activation='sigmoid'))  # Sigmoid activation for binary classification

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary to check architecture
model.summary()

# Add EarlyStopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stop]
)

# Evaluate the model on the validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {validation_accuracy * 100:.2f}%")


Found 267 images belonging to 2 classes.
Found 149 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,396,609 (169.36 MB)

 Trainable params: 44,396,609 (169.36 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 91s 7s/step - accuracy: 0.5758 - loss: 1.6766 - val_accuracy: 0.5503 - val_loss: 0.6914
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 209ms/step - accuracy: 0.5022 - loss: 0.6941 - val_accuracy: 0.5369 - val_loss: 0.6795
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 227ms/step - accuracy: 0.5208 - loss: 0.6932 - val_accuracy: 0.4832 - val_loss: 0.6843
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - accuracy: 0.5339 - loss: 0.6906 - val_accuracy: 0.6510 - val_loss: 0.6650
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 250ms/step - accuracy: 0.6013 - loss: 0.6771 - val_accuracy: 0.7315 - val_loss: 0.6518
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 205ms/step - accuracy: 0.5125 - loss: 0.6994 - val_accuracy: 0.7248 - val_loss: 0.6346
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 209ms/step - accuracy: 0.6114 - loss: 0.6535 - val_accuracy: 0.6174 - val_loss: 0.5414
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 204ms/step - accuracy: 0.6312 - loss: 0.5810 - val_accuracy: 0.8322 - val_loss: 0.4780
Epoc

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the ImageDataGenerator for the test set (no augmentation, just rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Set up the test generator
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Malaria/test',  # Path to your test directory
    target_size=(224, 224),   # Resize images to 224x224 (VGG16 input size)
    batch_size=32,            # Batch size
    class_mode='binary',      # Assuming binary classification (Infected/Uninfected)
    shuffle=False             # Don't shuffle for evaluation (important for consistent results)
)


Found 134 images belonging to 2 classes.


In [8]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

# Get the predictions for the test set
test_labels = test_generator.classes
test_predictions = model.predict(test_generator, verbose=1)
test_predictions = np.argmax(test_predictions, axis=1)

# Classification Report (Precision, Recall, F1-Score)
print("Classification Report:")
print(classification_report(test_labels, test_predictions))


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.7399 - loss: 0.4471
Test Accuracy: 78.36%
Test Loss: 0.4175
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81        91
           1       0.00      0.00      0.00        43

    accuracy                           0.68       134
   macro avg       0.34      0.50      0.40       134
weighted avg       0.46      0.68      0.55       134



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
